In [ ]:
#Program to load and preprocess Spotify audio features dataset

import pandas as pd
import numpy as np
import sklearn as sk
import spotipy
import time
import os

df = pd.read_csv('../data/raw/spotify_audio_features.csv')

# Strip any whitespace from column names (common issue)
df.columns = df.columns.str.strip()

# Chosen relevant features for analysis
features = ['key', 'tempo', 'acousticness', 'valence', 'instrumentalness',
            'mode', 'energy', 'danceability', 'loudness']


tracks_info = df[features]
print(tracks_info.head())
